In [8]:
import sys
import glob
import os
import numpy as np
from tqdm import tqdm
import pandas as pd
import plotly.express as px
sys.path.append("../")
from bengali_asr.dataset.encoder_decoder_dataset import load_audio
from IPython.display import Audio, display

In [3]:
from bnunicodenormalizer import Normalizer 
bnorm=Normalizer()

def normalize(sentence):
    _words = [bnorm(word)['normalized']  for word in sentence.split()]
    return " ".join([word for word in _words if word is not None])

In [4]:
old_df = pd.read_csv("/app/dataset/train_data_subset.csv")

In [5]:
new_df = pd.read_csv("/app/openASR/utt_spk_text.tsv",sep="	",header=None)
new_df.columns = ["id","client","sentence"]
new_df.id = new_df.id.apply(lambda x: x+".mp3")

In [12]:
sizes = []
for f in tqdm(new_df.id):
    sizes.append(np.load(os.path.join("/app/dataset/train_numpy_16k",f.replace(".mp3",".npy"))).shape[0]//320)

100%|██████████| 127565/127565 [00:29<00:00, 4331.61it/s]


In [19]:
norm_sentence = []
for sentence in tqdm(new_df.sentence):
    norm_sentence.append(normalize(sentence))

100%|██████████| 127565/127565 [03:38<00:00, 582.63it/s] 


In [ ]:
new_df.sentence = norm_sentence

In [23]:
new_df = new_df.dropna()

In [24]:
new_df["audiolen"] = sizes
new_df["tokenlen"]  = new_df.sentence.apply(len)

In [28]:
new_df = new_df[new_df.audiolen>=2*new_df.tokenlen]

In [35]:
VOCAB = {}
for x in new_df.sentence:
    for i in x:
        if i not in VOCAB:
            VOCAB[i]=0
        VOCAB[i]+=1

In [44]:
OLD_VOCAB = ['০', '১', '২', '৩', '৪', '৫', '৬', '৭', '৮', '৯','ও', ' ', 'ব', 'ল', 'ে', 'ছ', 'আ', 'প', 'ন', 'া', 'র', 'ঠ', 'ি', 'ক', '!', 'ো', 'ম', 'হ', 'ষ', '্', 'ট', 'গ', 'ত', 'চ', '?', 'ু', 'ঝ', ',', 'এ', 'স', 'থ', '।', 'শ', 'য', '়', 'ী', 'ধ', 'ঙ', 'ভ', 'জ', 'ই', 'দ', 'খ', 'ফ', 'ং', 'উ', 'ণ', 'অ', 'ঁ', 'ড়', 'য়', 'ঢ', 'ড', '-', 'ূ', 'ঘ', 'ৃ', 'ঞ', '‘', '’', 'ৈ', '"', '—', 'ৌ', 'ৎ', 'ঃ', ';', 'ঐ', 'ঈ', 'ঊ', '–', "'", 'ঋ', ':', '/', 'ঢ়', 'ঔ', '.', '“', '”']


In [45]:
{ x for x in VOCAB if x not in OLD_VOCAB}

{'\u200d'}

In [36]:
pd.concat([new_df,old_df]).to_csv("/app/dataset/train_data_with_openasr.csv",index=False)

In [3]:
tdf = pd.read_csv("/app/dataset/train_data_with_openasr.csv")

/tmp/ipykernel_1018494/1319503275.py:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  tdf = pd.read_csv("/app/dataset/train_data_with_openasr.csv")


In [54]:
new_df.sample(10)

,id,client,sentence
28045,398afb395c.mp3,e55de,গেতাফে ডিফেন্ডার আনহেল লাতিফা
124173,fbfcda42fc.mp3,7351e,উভয় পক্ষের প্রস্তাব
31875,3e8494fba1.mp3,b8ed8,বউ ঠাকুরানীর হাট অবলম্বনে
15333,2af32b541e.mp3,1a2a1,প্রথম থেকেই তারা চেয়েছেন
58580,697056f878.mp3,39293,৭৩ পুরুষ এবং ২৭ নারী
37877,4688c815d5.mp3,287bd,৫১ হাজার মেগাওয়াট
67006,75732de572.mp3,bcaa8,আলী আরশাদ জানান
34123,413e816374.mp3,4c555,দ্বিতীয় মুদ্রণ প্রকাশিত হয়
98171,a6d6613c96.mp3,4c555,কোনো নেতৃপদ তো দূরের কথা
14747,2a464b3ee8.mp3,85430,হার্ভার্ড কলেজে ভর্তির পর


In [59]:
display(Audio(np.load("/app/dataset/train_numpy_16k/4688c815d5.npy"), rate=16000))